# w261 Final Project - Airline Flight Delay Prediction

# Table of Contents

__Section 1__ - Question Formulation

__Section 2__ - EDA & Discussion of Challenges

__Section 3__ - Feature Engineering

__Section 4__ - Algorithm Exploration

__Section 5__ - Algorithm Implementation

__Section 6__ - Conclusions

__Section 7__ - Application of Course Concepts

##Section 1 - Question Formulation

From the NEXTOR *Total Delay Impact Study: A Comprehensive Assessment of the Costs and Impacts of Flight Delay in the United States* we can start to draw a picture of the heavy economic impact air travel delays can have. The study analyzed all US domestic flights in 2007 and found that total cost to the United States was estimated to be around $31.2 billion dollars. The costs are primarily attributed to the direct loss of profits and increased expenditures attributed to managing personnel, rerouting flights, maintenance, flight cancellations, etc. However, the existence of delays also have significant estimated welfare loss due to lowered demand as well as indirect trickle-down econommic impact on limiting potential travel and secondary industry stimulation (hospitality, entertainment, etc.). 

Machine learning algorithms have become much more extensively used to optimize airline and airport operations after evaluating factors impacting flight delays. Pyrgiotis et al. modeled propagation within an airport network, measuing impact of delays from one airport to another. The model iterated across (1) a queuing model computing delays at individual airports, and (2) delay propagation algorithm which continuously updates schedules and demand to all airport in the network off of the queuing model. Simaiakis et al. looked into potentially reducing taxi-times by modeling a queuing system for departing aircraft. Their work was further extended and developed using more robust datasets and statistical methods by Clewlow et al. More recently, Gopalakrishnan et al. compared various machine learning algorithms in prediction delayed flights -- though failed to consider neural networks and decision trees. 

While the primary stakeholders can be considered entities such as the Federal Aviation Administration (FAA), we can also consider traveling clientele and even federal agencies to have a key interest in accurate airline delay predictions. Mitigating the economic losses from unexpected delays will alleviate costs incurred by the airlines, increase customer satisfaction, and result in positive externalities thanks to efficient shipping of goods/services/individuals. Optimizing flight operations will also help mitigate the vast amounts of pollution created from aircraft -- taxi-out operations are responsible for 45,000 tons of carbon monoxide emissions in the US (2007).    

![](https://www.researchgate.net/profile/Jorge-Soares-12/publication/315382748/figure/fig4/AS:649756581826563@1531925449831/Data-model-of-the-flight-delay-prediction.png)

***

####Project Goal:   
>Predict flight departure delays two hours prior of scheduled departure (where a delay is defined as a minimum 15-minute delay with respect to the planned departure time)

**Questions to Consider:**
- What variables are known to us two hours prior to scheduled departure? (Avoid using highly correlated variables such as `DELAY` related columns as they bleed out of the allowed time frame)
- How shall we address messy/missing values?
- Are these high impact attributes impactful or simply contributing to data leakage?
- What additional insights/features can we draw out of the available data?

Model accuracy will help us determine how well our model does in classifying airlines as delayed or not delayed. However, knowing that our classes are imbalanced (majority not delayed flights) we will need to take good care to check metrics of precision and recall especially for delayed flights. The emphasis is on correctly identifying delayed flights and it would not be productive if we (worst case) had a model that predicted on-time for every single flight. (This would actually give us a model accuracy of 77.6%).  A meaningful value of precision and recall would be >80%, where delayed flights are correctly predicted for the most part and airlines could make the appropriate manuevers to mitigate the issue.

***

*Citations:   
M. Ball, et al. "Total Delay Impact Study: A Comprehensive Assessment of the Costs and Impacts of Flight Delay in the United States." 2010   
R. Clewlow, I. Simaiakis, and H. Balakrishnan, “Impact of arrivals on departure taxi operations at airports,” 2010.   
N. Pyrgiotis, K. M. Malone, and A. Odoni, “Modelling delay propagation within an airport network,” Transportation Research Part C: Emerging Technologies, vol. 27, pp. 60–75, 2013.   
I. Simaiakis and H. Balakrishnan, “Queuing models ofairport departure processes for emissions reduction,” in AIAA Guidance, Navigation and Control Conference and Exhibit, vol. 104, 2009.   
K. Gopalakrishnan and H. Balakrishnan, “A comparative analysis of models for predicting delays in air traffic networks,” in USA/Europe Air Traffic Management Seminar,2017.*

##Section 2 - EDA & Discussion of Challenges

**Missing Values:** 

Explore the available features and cut down on irrelevant ones to reduce the load on our models. Initial analyses of the data indicated that many data points, especially in the weather data set were blank.  The first such pass over the data was carried out by reading in the data, counting blanks per column via user-defined helper function.  Viewing columns having a number of blank values led to many columns being dropped. (See Feature Engineering - Final Data Processing Pipeline)

**Imbalanced Data:**   
As noted earlier when exploring our evaluation metric and dependent variable, the available data for our two classes (delayed and not delayed flights) is very unbalanced. This can lead to heavy biases in determining our model efficacy and notions of down/up-sampling are important to consider in our model implementations.

__6 Month__
![](https://snipboard.io/TJyukY.jpg)

__Full Data__
![](https://snipboard.io/VraAW5.jpg)

**Distribution of Numeric Values:**

For the 6 months data, observed the frequency distribution of various numeric variables. Note that this dataset only includes flights originating from two airports (as shown in ORIGIN_AIRPORT_ID and ORIGIN_STATE_FIPS). Among these plots, we can see DEP_DELAY, TAXI_IN, and TAXI_OUT have heavily right-skewed distributions. MONTH, DAY_OF_MONTH, WHEELS_ON, and WHEELS_OFF are fairly normally distributed. ARR_TIME and DEP_TIME are also fairly focused around daylight hours as one would expect.

![](https://snipboard.io/pJYyCE.jpg)

**Temporal Departure Delay Trends:**

For the temporal metrics that are easily accessible as soon as the flights are scheduled, we plotted the average departure delays across the 6 month data. For the two airports we can see distinct differences in delays between airports (ORD tends to be much higher). There are also distinct differences in temporal/seasonal trends:
- Months of March-May tended to have the least delays - likely due to less demand and scheduled flights
- Spikes in Delays tended to be most extreme on weekends and the least on the extreme ends of weekdays
- Flights prior to 5:00 AM were also the most delayed

![](https://snipboard.io/YtpyHh.jpg)

**Correlation Plots:**   
In the construction of linear models, collinearity can introduce instability by making multiple solutions valid.  Consider the process \\(y=x_1+x_2\\) where \\(x_2=2x_1\\).  This way, \\(y\\) can be wholly defined in terms of either \\(x_1\\) or \\(x_2\\):

$$ y=x_1+2*x_1 = 3x_1 $$
$$y=(1/2)x_2+x_2=(3/2)x_2 $$

Assuming ideal data with no noise, each of these equations is an equally valid regression, but the first equation gives the result \\((3=x_1, 0=x_2)\\) and the second \\((0=x_1, 3/2=x_2)\\).  However, these two solutions define a line themselves and show that \\((1=x_1,1=x_2)\\) being on the line is also a solution.  In fact any other point on the line \\(x_2=-0.5x_1+3/2\\) is also a solution.  The spectrum of solutions on the line makes optimization and finding a unique solution difficult, solutions with great or impossible and this motivates some analysis to find and eliminate collinear data.

To aid in collinearity detection, a correlation matrix was generated and plotted.  A correlation matrix is generated by taking each of the feature vectors and finding their pairwise correlation values.  The matrix is a square matrix whose rows are the same in number as the number of features.  Additionally, the correlation between features indexed by i and j are placed into the cell indexed by i and j.  A correlation heatmap plot is created by coloring a grid of cells of the matrix according to the correlation values.  We noticed from the plot below that no two features were perfectly correlated.

![](https://snipboard.io/d2TlyK.jpg)

***

#### Additional Scalability Precautions

**Saving Dataset and Caching:**   
Frequent saving of our datasets as a Parquet file and caching transformed data allowed us to run code efficiently. Caching and saving reduces the need to reload the data each time it is being used. In particular, at the end of our data processing notebook we save our final joined table to parquet for subsequent model iterations and final model runs.


[EDA Visuals](https://github.com/kks37405/flight_delay_prediction/blob/main/EDA%20Visuals.ipynb)

## Section 3 - Feature Engineering

#### Main Approaches
**Final Data Pipeline:**

We built our data processing pipeline in the following manner.

Airlines Table:
- Drop columns with more than 1/3 null values
- Drop irrelevant and redundant columns (i.e. Airport name and Airport ID)

Weather Table:
- Retain only US records 
- Keep columns of weather records for FM15 report type (Aviation routine weather report)
- Retain relevant columns from un-nested weather data
  - __Mandatory Data__ and __Additional Data__ section from the FEDERAL CLIMATE COMPLEX DATA DOCUMENTATION were used as a guide to select features from Weather table.


Joins:
1. Join stations table with weather records using concatenated USAF and WBAN codes
2. Add helper table codes to map airport ICAO and IATA identification systems
3. Join airlines with weather by taking only the most recent weather station data at least two hours prior to flight planned departure time

Missing Values:
- Drop rows with missing value for our dependent metric `DEP_DEL15`
- Data Imputation:
  - Median - for skewed numeric attributes and discrete variables
  - Mean - for non-skewed numeric attributes

[Data Processing Pipeline](https://github.com/kks37405/flight_delay_prediction/blob/main/Data%20Preprocessing%20Pipeline.ipynb)

**Feature Selection by L1/L2 Regression:**

L1 and L2 regression, also known as “Lasso” and “Ridge” regression respectively, share the sum of squares of residuals in their cost functions.  However each of L1 and L2 regressions contain an additional distinct term that adds to their costs.  The Lasso cost function contains the term i||and the Ridge cost function contains the term ii2.  Here the  are the weights that are “learned” during a regression’s cost optimization routines.  This way the extra term for each of the L1 and L2 cost functions adds a cost depending on the sum of the absolute values of the weights (for L1) or the sum of their squares (for the L2).  In either case the  parameter is used to tune the regression’s penalization; if zero then OLS is used because the penalty term itself becomes zero.

In the L1 regression because the sum is used some weights are forced towards zero while others flourish away from zero.  In some cases, it may depend on any collinearity present and initial values in any optimization process. We used our L1 and L2 regression models to guide our feature selection for our final dataset.

[L1/L2 Regression](https://github.com/kks37405/flight_delay_prediction/blob/main/L1%20Regression.ipynb)

**Indexing and Encoding Categorical Values:**

To handle categorical values in the dataset, we applied indexing and one hot encoding to convert to numeric datatypes suitable for the algorithms.  These transformations were necesssarily depending on the algorithms.  For example, decision trees can handle categorical data while logistic regression could only handle numeric data.

**Down-sampling:**   
To balance out the dataset, we applied the approach of downsampling the majority class so the number of observations for on-time flights were roughly even with delayed flights.  Balancing the classes prevents the data from introducing a bias in the models, where it may just simply predict the majority class all the time.  Up-sampling the minority class would have been another approach if we had a very small dataset and each observation would be too valuable to discard.  But given that we had >10 million rows of data, down-sampling was a resource appropriate technique for our problem.

***

#### Alternative Approaches 

**Dimensionality Reduction with PCA:**   
The essential function of a Principal Component Analysis (PCA) is to take the matrix of data, rearrange and factor it so as to express it in terms of “principal components” which might reveal structure in the data.  In some sense, it’s not unlike considering the integer value 66 but in considering its binary equivalent 1000010 (2^6+2^1=64+2=66).

What PCA does is it expresses each datapoint in terms of “principal component vectors” and “principal component values”.  PCA can be interpreted as iteratively seeking vectors who define a span of greatest variability.  Once that vector is defined, the process can be considered as repeating iteratively until a vector is created for each feature dimension.  These “principal vectors” once defined can often reveal vectors which best separate labels.  This is because the vectors are defined by seeking bases of greatest variability.  For such reasons PCA is often used as a dimensionality reduction technique.

We subjected our weather data to PCA. After that, the resulting PCA vectors were input to a decision tree classifier trained on the data using the labels. We got an accuracy of 78% for a decision tree classifier and 88% with a random forest.

**New Features:**
- Rolling percentage of flights delayed in an airport that day (excluding current entry)
- Percentage of flights delayed in an airport that day in the hour block two steps prior to the block the current flight is contained in

Both of these features for consideration were ultimately excluded because we decided to avoid time-series dependent features. Later on in our algorithm implementation, we opted for a randomized test-train split solely dependent on features easily accessible and available for the model. That way we could still make reasonable predictions even if we applied the classifier on new data (in which case we may have no information on flight delays earlier in the day). 

[PCA + New Features](https://github.com/kks37405/flight_delay_prediction/blob/main/PCA%20%2B%20New%20Features.ipynb)

## Section 4 - Algorithm Exploration

**Screening Models:**  

Three tree based algorithms were trained on the 6 month dataset to serve as baselines to select a final algorithm for the full dataset.  The three tree algorithms were: decision tree, random forest, and gradient boosted decision tree.  The algorithms also underwent simple hyperparameter tuning during cross validation.

Of all the algorithms, it was expected that the gradient boosted decision tree would produce the highest score given the model's complexity.  A likely drawback of the gradient boosted decision tree however, is that it can be prone to overfitting or high variance.  A random forest would less likely to overfit than gradient boosted decision tree given its complexity but may have higher bias.  To prevent overfitting, we applied cross validation with hyperparameter tuning to find the model with the best score.

**Choosing our best model:** 

The evaluations from the three models on the 6 month test set is shown below.  In terms of Precision and Recall score, a gradient boosted decision tree performed the best and was chosen for the final algorithm.  A notebook detailing the implementation of the algorithms can be found here: 

[Algorithm Exploration](https://github.com/kks37405/flight_delay_prediction/blob/main/Algorithm%20Exploration-6m-UnderSample.ipynb).   

![Precision and Recall](https://snipboard.io/r80Juc.jpg)

##Section 5 - Algorithm Implementation

**Decision Trees:**   
Decision trees classify data like a coin falling through a chute with gates whose sizes are tuned for the various denominations.  Each node in the tree defines an easy to interpret rule which is very helpful to understand the tree’s explainability and how it works and is used in classification. The tree construction process is relatively complex however. It relies on a recursive and “greedy” process to define rules used to classify the data.  The “greedy” term is used to indicate that the same process is used at each branch point’s creation without regard to what may be happening at any other branch or with any other data.  

As the tree is built the data at each node is subjected to an information-gain analysis based on “impurity” values. The impurity is a measure of data homogeneity. At each branch point, different binary splits are considered.  For each split considered the information gain is calculated as the impurity of the data at the node minus the impurities of the data at each subtree if the split were carried out.  This is detailed as in the equation below: 
$$IG(D,s) = Impurity(D)-\frac{N_{left}}{N} Impurity(D_{left})-\frac{N_{right}}{N}Impurity(D_{right}) $$

For our tree modeling we used the Gini Impurity. In our dataset there were two labels : DEP_DEL15 yes (1) and DEP_DEL15 no (0).  For an example consider 100 records evenly split with 50 labels of each.  If a binary split puts all of “yes” label on one side the remaining “no” we’d have zero impurity on the left, 0 impurity on the right and the branch under consideration would have impurity of 0.5 yielding a gain of 0.5.  In contrast supposing a split were to be non-informative and put 25 “yes” in both the left and right branches as well as 25 “no” in both branches too.  In that case the impurities in each branch would be the same and their weighted sum would equal the total impurity at the root and so information gain would be zero.  The Gini impurity is calculated as below:

$$\sum^C_{i=1} f_i (1 - f_i) $$
where f is frequency of label i at a node and C is the number of unique labels

**Gradient-Boosted Trees** are ensembles of decision trees, they iteratively train DTs in order to minimize a loss function.

**Random Forests** are ensemble trees, they combine decision trees in order to reduce the risk of overfitting.

#### 5a Decision Trees - A Small Toy and Exercise

A process is used to simulate weather that is either "Sunny" or "Not Sunny".  Temperature is allowed to range from 50 to 100.  If temperature is an odd number then there is precipitation.  Finally, if the temperature is greater than or equal to 75 and if there is no precipitation then the weather is set as sunny.  For demonstration purpposes, an additional non-informative random boolean value "rand_bool" is added.  Being random it carries no information and is not expected to help in terms of information gain.

In our weather dataset we had precipitation data.  One way we could have worked with it in terms of feature engineering is to set it as a binary variable "Precipitation" or "No Precipitation".  This toy data, having only boolean data therefore presents a hypothetical avenue of that kind of feature engineering.

In [0]:
gen_data=list()
import random

#temp ranges from 50 to 100
for temp in range(50,101):
    sunny=False
    temp_geq_75=temp>=75
    precip=temp%2==0
    sunny=temp_geq_75 and not(precip)
    rand_uni=random.random()
    rand_bool=False
    if rand_uni>=0.5:
      rand_bool=True
    row=[temp_geq_75,precip,rand_bool,sunny]
    gen_data.append(row)
import pandas as pd

toy_df = pd.DataFrame(gen_data, columns = ['temp_geq_75','precip','rand_bool','sunny'])
print("The first few records")
print(toy_df.head())
print("The last few records")
print(toy_df.tail())

The first few records
 temp_geq_75 precip rand_bool sunny
0 False True False False
1 False False True False
2 False True False False
3 False False False False
4 False True False False
The last few records
 temp_geq_75 precip rand_bool sunny
46 True True False False
47 True False True True
48 True True True False
49 True False True True
50 True True False False

##### Tree-Related Routines

A routine to compute Gini-Impurity from a list of booleans is written. A utility to return the most frequent boolean is written. Finally, a function to find information gain if a data frame is split on a column is also provided.  In addition, a recursive routine to generate a decision tree (in the form of a dict) is written which invokes aformention dependent routines. Once a column has been used it cannot be re-used. Once there are no columns to consider, then the result of the tree is the most frequent label seen in the resulting data.

In [0]:
def calcImpurity(boolean_list):
    """Compute GINI impurity"""
    #https://spark.apache.org/docs/2.2.0/mllib-decision-tree.html#node-impurity-and-information-gain
    num_labels=2
    true_data=[b for b in boolean_list if b==True]
    false_data=[b for b in boolean_list if b==False]
    true_freq=float(len(true_data))/float(len(boolean_list))
    false_freq=float(len(false_data))/float(len(boolean_list))
    freqs=[false_freq,true_freq]
    impurity=sum([f_i*(1.0-f_i) for f_i in freqs])
    return impurity

def getMostFrequent(boolean_list):
    """return the most frequent boolean (True or False)"""
    count_true=len([b for b in boolean_list if b==True])
    count_false=len([b for b in boolean_list if b==False])
    if(count_true>=count_false):
        return True
    else:
        return False

def colSplitIG(df,col_name,target_col):
    """if a dataframe whose label/target is indicated, but split on the given column , return the information gain"""
    #get the splits data structures
    left_df=df[df[col_name]==False]
    right_df=df[df[col_name]==True]
    left_df_labels=left_df[target_col].tolist()
    right_df_labels=right_df[target_col].tolist()
    here_labels=df[target_col].tolist()
    
    #compute gain
    impurity_here=calcImpurity(here_labels)
    #left 
    r_left=float(len(left_df_labels))/float(len(here_labels))
    imp_left=calcImpurity(left_df_labels)
    left_term=r_left*imp_left
    #right
    r_right=float(len(right_df_labels))/float(len(here_labels))
    imp_right=calcImpurity(right_df_labels)
    right_term=r_right*imp_right
    #final computation
    info_gain=impurity_here-left_term-right_term
    return info_gain

def returnTree(df,target_col,cols_to_consider):
    """recursively implement a decision tree algorithm.  Never re-use a column"""
    #the base case for the recursion is when len(cols_to_consider)==0 ; just return the most common target label
    the_dict=dict()
    if(len(cols_to_consider)==0):
        the_dict['return']=str(getMostFrequent(df[target_col].tolist()))
        return the_dict
    #in the recurive case, keep track of info. gain depending on the column splits
    #then once we know what column is used (whose split results in greatest gain)
    #subtract that column from the list considered. This 'trimmed' list of columnsis passed in 
    #the recursive call as cols_to_consider so that no column is re-used
    cols_to_skip=list()
    the_igs=list()
    #consider splitting on each column and compute the info. gain
    for col_name in cols_to_consider:
        ig=colSplitIG(df,col_name,target_col)
        the_igs.append(ig)
    #find the column whose split led to max gain and remove it from a list
    max_ig=max(the_igs)
    max_ig_idx=the_igs.index(max_ig)
    max_ig_col_name=cols_to_consider[max_ig_idx]
    cols_to_skip=[c for c in cols_to_consider if c!=max_ig_col_name]
    #set up for a recursive vall
    dict_key_f=str(max_ig_col_name)+'==false'
    dict_key_t=str(max_ig_col_name)+'==true'
    left_df=df[df[max_ig_col_name]==False]
    right_df=df[df[max_ig_col_name]==True]
    #call recursively and return
    the_dict[dict_key_f]=returnTree(left_df,target_col,cols_to_skip)
    the_dict[dict_key_t]=returnTree(right_df,target_col,cols_to_skip)
    return the_dict

##### Build A Tree

Use the code to generate a decision tree!  As seen in the toy tree, the first split is on precipitation which is most informative.  Finally, towards the leaves, the last splits are on the "rand_bool" , which, due to its random nature, is least informative.  Its non-informative nature puts its split at the leaves because it gives the least information gain!

In [0]:
import json
#The target column is 'sunny'
target_col='sunny'
#use the other columns
cols_to_consider=['temp_geq_75','precip','rand_bool']
#build and print a tree!
toy_tree=returnTree(toy_df,target_col,cols_to_consider)
print(json.dumps(toy_tree,indent=4))

{
 "precip==false": {
 "temp_geq_75==false": {
 "rand_bool==false": {
 "return": "False"
 },
 "rand_bool==true": {
 "return": "False"
 }
 },
 "temp_geq_75==true": {
 "rand_bool==false": {
 "return": "True"
 },
 "rand_bool==true": {
 "return": "True"
 }
 }
 },
 "precip==true": {
 "temp_geq_75==false": {
 "rand_bool==false": {
 "return": "False"
 },
 "rand_bool==true": {
 "return": "False"
 }
 },
 "temp_geq_75==true": {
 "rand_bool==false": {
 "return": "False"
 },
 "rand_bool==true": {
 "return": "False"
 }
 }
 }
}

## Section 6 - Conclusions

__Final Gradient Boosted Tree Result:__ 6 -12 hrs.

[Final Gradient Boost](https://github.com/kks37405/flight_delay_prediction/blob/main/Full-Model.ipynb)

```
'ROC AUC': 0.7043538319027595, 'PR AUC': 0.6493648690779376
              precision    recall  f1-score   support
         0.0       0.66      0.75      0.70   1139216
         1.0       0.63      0.54      0.58    929962
    accuracy                           0.65   2069178
   macro avg       0.65      0.64      0.64   2069178
weighted avg       0.65      0.65      0.65   2069178
```

|                         Model: Dataset                           |Recall|Precision|ROC AUC|  PR AUC  |
|------------------------------------------------------------------|------|---------|-------|----------|
|__DT__ (depth=0, trees=1): __6 month__                            |0.78  |0.603    |0.5    |**0.22**  |
|__RF__ (depth=6, trees=20): __6 month__                           |0.78  |0.603    |0.68   |**0.40**  |
|__GBT__ (depth=6, trees=30): __6 month__                          |0.80  |0.772    |0.72   |**0.48**  |
|__DT__ (depth=10, trees=1327): __6 month Undersample__            |0.66  |0.66     |0.60   |**0.54**  |
|__RF__ (depth=10, trees=30, bins =80): __6 month Undersample__    |0.65  |0.65     |0.71   |**0.64**  |
|__GBT__ (depth=10, trees=50): __6 month Undersample__             |0.68  |0.68     |0.74   |**0.66**  |
|__GBT__ (depth=10, trees=50): __Full Undersample__                |0.65  |0.65     |0.70   |**0.65**  |

##### Scalability

_Data transformation and pre-processing:_
- Our data pipline proved to be scalable throughout the transformation and pre-processing workflow (from the 6 month data to the full dataset). The main factor in its scalability was frequent use of saves into parquet, this allowed us to avoid build up of transformation steps. 
- Our join proved very scalable as it completed in a matter of minutes on the smaller 6 month data as well as the full dataset.
- In order to improve scalability we also reduce the amount of data we transformed from the start by removing unnecessary columns and irrelavent columns. Thus we did not waste resources transforming data that was not needed. 
- We also cached our dataset before running our model to improve our performance when running on a larger dataset.

_ML_
- When using `CrossValidator` we noticed a major dip in performance even when running on the smaller 6 month dataset. To improve performance we utilized the `parallelism` parameter with `CrossValidator` to set the number of threads to use when running our models in parallel. 
- Early on we used the `pipeline` as the estimator inside `CrossValidator`which executes the entire pipline for every parameter combination fold. With this configuration, our inital Gradient Boosted models took almost half a day to complete. However, for our final model, we set the model specification as the estimator inside the `CrossValidator` since only the model was being tuned in this case and not the `pipeline` to improve performance.

***

*Citations:   
Databricks. “How to Speed up Cross-Validation.”  2020, kb.databricks.com/machine-learning/speed-up-cross-validation.html.*

##### Evaluation Metrics

- Recall : Percentage of true positives identified correctly. Out of all the actual 1s, what percent did we predict as 1s.
$$Recall = \frac{True Positive}{True Positive + False Negative}$$
- Precision : Allows us to measure the correct proportion of positive identifications. Out of all our predictions of 1s, what percent were truly 1s.
$$Precision = \frac{True Positive}{True Positive + False Positive}$$
- ROC AUC : Provides insight on how our good our model is at distinguishing between a 0 (Departure delayed less than 15 minutes) and 1 (Departure delayed 15 minutes or more). The higher the ROC AUC, the better the model is at distinguishing between the two.
- PR AUC : Using precision to provides insight on how many of our predicted positives are true positives. The higher the PR AUC, the better the model is at labeling 0s as 0s and 1s as 1s. This metric is more informative for our case as our data has a high class imbalance between flights between a 0 (Departure delayed less than 15 minutes) and 1 (Departure delayed 15 minutes or more).


Our final model, the Gradient Boosted Tree, does a decent job at labeling 0 (Departure delayed less than 15 minutes) identified by its 0.75 recall, but is almost flipping a coin to predict 1 (Departure delayed 15 minutes or more), identified by its 0.54 recall. Our ROC AUC mostly likely as high as it is due the models ability to better predict 0s.

In terms of precision, our model is just as good at predicting true negatives (0s as 0s) as it is at predicting true positives (1s as 1s). However, the precision is only marginally better than a coin toss. The metrics of our final model are consistent with those that were displayed in its 6 month counter parts. However, The reason why the 6 month Gradient Boosted Tree performs better than the full Gradient Boosted Tree, is due to the difference in imbalance between 6 month and the full data. The 6 month data has 77.6% 0s and 22.4% 1s compared to 81.8% 0s and 18.2% 1s.

Even though most of our metrics for our final have a lower score than its predecessors, our PR AUC is in line with the 6 month data and has highly improved which means compared to the models that came before it (especially ones that did not have undersampling), our final model is the best at predicting 1s (Departure delayed 15 minutes or more). A major factor in our improved targeted measure, PR AUC, is due to our addition of under sampling. Using under sampling we created a closer distribution of labels, 55% 0s and 45% 1s.

Given the business problem, our model performs poorly at predicting flight delays. Due to the imbalanced labels, our model had a hard time correclty predicting delays. However, we improved our metrics from our initial decision tree has highlighted by the PR AUC score through use of undersampling.

***

*Citations:   
J. Davis, M. Goadrich. "The Relationship Between Precision-Recall and ROC Curves." 2006.*

##### Considerations for Improvement
- Undersampling helped our final model improve in its label prediction, however, to further improve our model we can consider oversampling. Oversampling is the process of duplicating the minority class, in this case departure delay, in order to have a more even distribution of classes.
- Imputation during pre-processing was done through custom code. Using Spark's `Imputer` to complete missing values, we can improve scalability. 
- Consider creating new features from our dataset.

## Section 7 - Application of Course Concepts

##### Model Assumptions

Decision trees are non-parametric methods and make no assumptions on the distribution of the underlying data. The same applies for random forests where we have an ensemble of decision trees. These methods can handle skewed and multi-model data, as well as categorical data (both ordinal or non-ordinal).

Logistic regression has various assumptions such as linearity in the logit of continuous variables, no multicollinearity, independent errors, and no strong outliers in the data.

##### Hyperparameter Tuning, Complexity, Bias, and Variance

In algorithm exploration, we practiced the concepts hyperparameter tuning, complexity, bias and variance in decision trees.  When building the tree algorithms, we had to balance complexity and generalization through hyperparameter tuning of depth (and number of trees in gradient boosted trees).  Though a deeper tree would lead to higher complexity and low bias, it could become prone to overfitting or high variance.  We employed random forests to address the issue of high variance in decision trees however the model complexity was not enough to produce meaningful results.  Ultimately we used gradient boosted trees and prevent overfitting by hyperparameter tuning for the highest score in the validation set.

##### Model training time 

Training the models on the full dataset of > 20 million rows was a time intensive process and to produce results quickly we screened models and features with a smaller dataset.  The strategy of using a smaller dataset first allowed us eliminate models and focus the lengthy training times on a model that would have the most potential.  The stategy also allowed us to perform unit testing on our code before running the full dataset.  A limitation of using a smaller dataset first is that it may not fully represent the full dataset. To accomodate this limitation, we ensured both datasets had a similar distribution of classes.

##### Cluster Managerment

While training our dataset and performing cross-validation, performance took a big dip as our cross-validator was executing the entire pipline for every parameter combination fold. When clusters are overloaded, runtime drastically increases turning code that would run in 3 minutes to 21 minutes. This is expounded when running multiple notebooks and models at the same time on the larger datasets. In a certain case when a cluster had running for more than 24 hours, we ran into errors and reduction in performance. Considering cluster health and restarting when it had been active for a long period of time noticeably increase performance.

##### Big Data Piplines

During our transformation and pre-processing phase we applied concepts from Big Data Pipelines to maximize performance and scalability. Using Parquet checkpoints kept metadata about the data in order to compute some calculations extremely quickly such as row counts. Use of UDFs were avoided as they are typically much slower than built-in Spark functionality because they need to serialize and deserialize the data for every row that the function is applied to.